In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
from transformers import TextDataset,LineByLineTextDataset,DataCollatorForLanguageModeling
from transformers import AutoTokenizer

C:\Users\asus\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
translated_dataset = pd.read_csv('translated_romance_2.csv')
translated_dataset

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,f3643541-c339-4377-851d-144ea400d551,Ibu tiri,Eric dan istrinya memiliki seorang putri berna...,Istri Eric meninggal.,Eric dan Meg sangat sedih.,Eric bertemu dengan seorang wanita dan menikah...,Meg sekarang senang dengan ibu tirinya yang baru.
1,d61b69c7-7eca-4099-ade2-1a7e5a014a13,Riasan yang buruk,Marnie akan menikah.,"Untuk pernikahannya, dia perlu makeup selesai.",Akibatnya dia menyewa penata rias untuk memban...,"Ketika penata rias selesai, Marinir melihatnya.","Dia ngeri dan akhirnya memecatnya,"
2,3ea38bec-1101-4a64-86fa-948904d33a2b,Bersatu kembali di reuni,Tom selalu bertanya -tanya apakah dia pernah m...,Dia telah memutuskan untuk menikahi orang lain...,"Sekarang, dia bercerai dan akan datang ke reun...",Tom dan Sue menghabiskan sebagian besar reuni ...,Kedua kekasih yang bersatu kembali telah bersa...
3,f085aab6-c13a-46d6-aca0-cf942b50577a,Keluarga baru,Jenn dan Bill jatuh cinta dan menikah.,"Beberapa bulan kemudian, Jenn mengetahui bahwa...",Bill sangat bersemangat dan tidak sabar untuk ...,Mereka mengetahui bahwa mereka memiliki bayi l...,"Begitu bayi itu lahir, mereka menamainya Theod..."
4,e19867fe-9b77-4a56-934a-9fd00d4c802e,Lone Girl,Dessie ingin bertemu pasangan baru.,Dia pergi melalui banyak situs kencan tidak be...,"Akhirnya, dia menyerah dan pergi ke perpustakaan.","Anehnya, dia bertemu dengan seorang pria tampa...","Setelah beberapa minggu, mereka bersama dalam ..."
...,...,...,...,...,...,...,...
933,6cb171ed-99f3-46f1-8d98-5dad776d9555,Wahyu Mike,Mike menunda meminta pacarnya untuk menikah de...,Dia telah mengatakan kepadanya bahwa dia ingin...,"Sekarang kuliah itu selesai, dia merasa tertek...","Sayangnya untuk Mike, keluar bahwa dia tidak i...",Sekarang Mike menyadari bahwa dia seharusnya j...
934,4de38ecd-39a3-4c7c-988e-93e6f8ec7705,Ultimatum Karen,Karen telah bersama pacarnya selama bertahun -...,Sekarang adik laki -lakinya bertunangan dan me...,Karen merasa sudah waktunya pacarnya untuk ber...,Dia memberinya ultimatum untuk menikah atau pe...,Pacarnya memilih untuk pergi.
935,7ce8f342-d590-40de-a312-aec4a638dcc9,Pernikahan atau kami putus,Janet dan John telah berkencan selama 10 tahun.,Janet selalu ingin menikah dengan John.,Tapi John tidak pernah ingin menikah.,Janet memberi tahu John bahwa mereka akan meni...,"Sayangnya untuk Janet, John memilih bahwa mere..."
936,31be3d5e-1c96-4c46-92e1-70bb1a888ee6,Bayi YouTube,Latoya dan Adam bertemu di acara YouTube.,Mereka berdua memiliki saluran YouTube sendiri.,Keduanya memiliki chemistry yang hebat dan mul...,Setahun kemudian mereka pindah bersama dan men...,Mereka telah bersama selama 5 tahun sekarang d...


In [3]:
tokenizer = AutoTokenizer.from_pretrained("indonesian-nlp/gpt2")
print("Sebelum")
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
print("Vocabulary size:", tokenizer.vocab_size)
print("Pad token ID:", tokenizer.pad_token_id)

special_tokens_dict = {'additional_special_tokens': ['<|firstoftext|>','<|lastoftext|>', '<|sentence1to2|>', '<|sentence2to3|>', '<|sentence3to4|>', '<|sentence4to5|>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print("Setelah")
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)  
print("Vocabulary size:", tokenizer.vocab_size)
print("Pad token ID:", tokenizer.pad_token_id)

Sebelum
['<|endoftext|>']
[50256]
Vocabulary size: 50257
Pad token ID: None
Setelah
['<|endoftext|>', '<|firstoftext|>', '<|lastoftext|>', '<|sentence1to2|>', '<|sentence2to3|>', '<|sentence3to4|>', '<|sentence4to5|>']
[50256, 50257, 50258, 50259, 50260, 50261, 50262]
Vocabulary size: 50257
Pad token ID: None


In [4]:
tokenizer = AutoTokenizer.from_pretrained("indonesian-nlp/gpt2")

# Your input sequences
sequences = ["This is a sequence.", "Another sequence.", "Yet another one."]

# Tokenize the sequences
tokenized_sequences = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# Print the padded sequences
print(tokenized_sequences)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.